### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

### Webbskraparen

In [6]:
start_time = time.time()

#Sparar request för page 1-200 i en lista
urlres = []
for x in range(200):
    res = requests.get("https://fi.trustpilot.com/review/flixbus.com?languages=all&page=" + str(x+1))
    urlres.append(res)
    
#Använder BeautifulSoup för att konvertera requesterna till BeautifulSoup element och sparar i soup-variabeln
#Appendar alla sidor ihop för att få alla fem sidor till en variabel
soup = BeautifulSoup(urlres[0].text, 'html.parser')

for x in range(499):
    soup2 = BeautifulSoup(urlres[x+1].text, 'html.parser')
    soup.html.append(soup2.html)
    
#print(soup)

titles = soup.findAll("h2", {"class": "typography_heading-s__f7029 typography_appearance-default__AAY17"})
titleClean = []
for row in titles:
    titleClean.append(row.text)
    
#titleClean

bodyText = soup.findAll("p", {"class": "typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn"})
bodyTextClean = []
for row in bodyText:
    bodyTextClean.append(row.text)
    

ratings = soup.findAll("div", {"class":"styles_reviewHeader__iU9Px"})
ratingClean = []
for row in ratings:
    ratingClean.append(str(row)[68])


location = soup.findAll("div", {"class":"typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua"})
locationClean = []
locationClean1 = []
for row in location:
    locationClean.append(str(row.find_all("span")))
    
for row in locationClean:
    locationClean1.append(row[7:9])


date = soup.findAll("div", {"class":"typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH"})
dateClean = []
dateClean1 = []
for row in date:
    dateClean.append(str(row.find_all("time")))
    
for row in dateClean:
    dateClean1.append(row[67:77])

#dateClean1
print("--- %s seconds ---" % (time.time() - start_time))

--- 335.5764138698578 seconds ---


### Skapar en pandas dataram

In [10]:
website = []

for row in titleClean:
    website.append("Trustpilot")

reviewsDF = pd.DataFrame( [titleClean,bodyTextClean,ratingClean,locationClean1,dateClean1,website], 
                         index=["Titel","Brödtext","Gradering","Plats","Datum","Webbsida"])

reviewsDF = reviewsDF.transpose()
reviewsDF.head(20)

,Titel,Brödtext,Gradering,Plats,Datum,Webbsida


In [12]:
reviewsDF.describe()

,Titel,Brödtext,Gradering,Plats,Datum,Webbsida
count,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
reviewsDF.to_csv('trustpilot_flix_bus.csv', index=False)